In [1]:
from bs4 import BeautifulSoup
import requests
from datetime import date, timedelta
import pandas as pd

import csv
import warnings
warnings.filterwarnings('ignore')

import collections
import re
import sys
import time

import pymorphy2
import numpy as np

В requests.get копируем url требуемого писателя 19-го века. Например Гоголь


In [2]:
tuttoarr2 = []
r  = requests.get("http://az.lib.ru/p/pushkin_a_s/text_0430.shtml",   )
data = r.text
soup = BeautifulSoup(data)
inputTag = soup.findAll("dl")

for link in inputTag:               
                    try:                        
                            title = link.findChildren("a",recursive=True)
                            for titlea in title: 
                               if titlea.has_attr('href') and not titlea.has_attr('class'):
                                   if 'comment' not in titlea['href']:
                        
                                        tuttoarr2.append(tuple((titlea['href'],titlea.text)))
                    except IndexError:
                        print('Rien')    

In [3]:
newarr=[]
for elem in range(1):
    r  = requests.get('http://az.lib.ru/p/pushkin_a_s/text_0430.shtml'  )
    data = r.text
    soup2 = BeautifulSoup(data)
    inputTag2 = soup2.findAll("div", {"align":"justify"})
    for link in inputTag2:

                        try:

                                title = link.findChildren("dd",recursive=True)
                                for titlea in title: 
                                   

                                            newarr.append(titlea.text)
                                            
                        except IndexError:
                            print('Rien')    

In [4]:
det = pd.DataFrame(newarr, columns=['text'])
det.to_csv('data/teramshort.csv', index=False)

In [5]:
det.describe()

,text
count,769
unique,768
top,\n
freq,2


In [6]:
def count_ngrams(lines, min_length=1, max_length=3):
    lengths = range(min_length, max_length + 1)
    ngrams = {length: collections.Counter() for length in lengths}
    queue = collections.deque(maxlen=max_length)

    def add_queue():
        current = tuple(queue)
        for length in lengths:
            if len(current) >= length:
                ngrams[length][current[:length]] += 1

    for line in lines:
        for word in tokenize(line):
            queue.append(word)
            if len(queue) >= max_length:
                add_queue()

    while len(queue) > min_length:
        queue.popleft()
        add_queue()

    return ngrams

In [7]:
def my_lemmatizer_ru(sent):
#     sent = ' '.join(sent)
    lemmatizer = pymorphy2.MorphAnalyzer()
    tokenized_sent = sent.split()
    return ' '.join([lemmatizer.parse(word)[0].normal_form
                    for word in tokenized_sent])

In [8]:
def tokenize(string):
    #re.findall(r'[А-Яа-я]+', string.lower()))
    return string.split(' ')

In [9]:
def print_most_frequent(ngrams, num=10):
    for n in sorted(ngrams, reverse=False):
        print('----- {} most common {}-grams -----'.format(num, n))
        for gram, count in ngrams[n].most_common(num)[:-num:-1]:
            print('{0}: {1}'.format(' '.join(gram), count))
        print('')

In [10]:
a = tokenize(det.text[0])

In [11]:
# ngrams = count_ngrams(det.text[0].split(' '))
ngrams = count_ngrams(a)
print_most_frequent(ngrams)

----- 10 most common 1-grams -----
Я: 345
что: 380
я: 430
с: 448
на: 466
--: 497
не: 599

  : 691
в: 824

----- 10 most common 2-grams -----
не мог: 29
я не: 29

   Стр.: 36
и с: 37
-- сказал: 40
 : 41
и не: 43
Марья Ивановна: 57

   Я: 63

----- 10 most common 3-grams -----
Петр Андреич! --: 9

   На другой: 9

   -- Не: 9

   -- Да: 9
На другой день: 10

   -- Что: 11
. . .: 11
В эту минуту: 14

   -- А: 14



In [12]:
old_corpus_word = []
old_corpus_freq = []
for i in range(0, 3):
    for k, v in ngrams[i+1].items():
        old_corpus_word.append(' '.join(k))
        old_corpus_freq.append(v)

In [13]:
def down_file(file):
    a = []
    with open(file, mode='r', encoding='utf-8') as infile:
        reader = csv.reader(infile, delimiter='\n')
        for row in reader:
            a.append(row)
    return a

In [14]:
full_dic = down_file('data/k3.txt')

In [15]:
word_dic = []
def_dic = []
for i in range(0, len(full_dic)):
    word_dic.append(full_dic[i][0].split(' – ')[0].lower())
    def_dic.append(full_dic[i][0].split(' – ')[1])

In [20]:
# WordDescription.create(id: 1, word: "текст", description: "описание слова текст")
output = []
# output_desc = []
for i in range(0, len(word_dic)):
    for j in range(0, len(old_corpus_word)):
        if ''.join(word_dic[i].split(' ')) == ''.join(old_corpus_word[j].split(' ')) :
            output.append([' '.join(old_corpus_word[j].split(' ')), 
                           ' '.join(def_dic[i].split(' '))])
            #output_desc.append('WordDescription.create(id: '+str(i)+', word: \''+
            #                  ' '.join(old_corpus_word[j].split(' '))+'\''+', description: \''+
            #                  ' '.join(def_dic[i].split(' '))+'\'')
            print(old_corpus_word[j], def_dic[i])

алтын старинная русская монета достоинством в три копейки.
армяк крестьянский кафтан; верхняя долгополая одежда в виде халата или кафтана из сукна или грубой шерстяной материи.
артикул воинский устав.
барин 1. Дворянин, землевладелец, помещик;2. Господин, владелец, хозяин.
в случае в милости у влиятельного лица, под покровительством влиятельных лиц. Ср. у И. А. Крылова
галоп популярный бальный танец XIX века под музыку размером в две четверти (по счёту раз, два).
дабы чтобы; для того, чтобы.
камер-лакей придворная должность: старший лакей при царском дворе.
капрал лицо, имевшее первый после рядового военный чин; унтер-офицер.
кибитка зимняя крытая повозка на полозьях.
лучина тонкая длинная щепка, использовавшаяся для освещения помещения.
мусье (искаж. франц. monsieur)
облучок сидение для кучера в повозке.
оный тот; тот самый.
оттоле оттуда.
подать подушный налог, взимавшийся с крестьян.
поручик младший офицерский чин.
трактир гостиница с рестораном; ресторан низшего разряда.
холоп двор

In [24]:
for i in range(0, len(output)):
    for j in range(0, len(a)):
        if output[i][0] == a[j]:
            s = "<a class='show_description_link' data-remote='true' data-method='get' href='/dhh/word.json?word="+str(i)+"'>"+a[j]+"</a>"
            a[j] = s

In [25]:
output[19][0] == ' час от часу '

False

In [26]:
b = []
b.append(' '.join(a))

In [27]:
output = pd.DataFrame(output)
output.to_csv('data/output.txt', index=True, header=False)


b = pd.DataFrame(b)
b.to_csv('data/text.txt', index=False, header=False, encoding='utf-8')

In [31]:
output[0]

0           алтын
1           армяк
2         артикул
3           барин
4        в случае
5           галоп
6            дабы
7     камер-лакей
8          капрал
9         кибитка
10         лучина
11          мусье
12        облучок
13           оный
14         оттоле
15         подать
16        поручик
17        трактир
18          холоп
19    час от часу
20           штоф
21          ямщик
Name: 0, dtype: object

In [32]:
output_desc = []
for i in range(0, len(output[0])):
    output_desc.append('WordDescription.create(id: '+str(i)+', word: \''+
                              output[0][i]+'\''+', description: \''+
                              output[1][i]+'\'')

In [33]:
output_desc = pd.DataFrame(output_desc)
output_desc.to_csv('data/output_desc.txt', index=False, header=False)